In [1]:
from datasets import load_dataset
import os
import json 

livecodebench = load_dataset("livecodebench/code_generation_lite", version_tag="release_v5", trust_remote_code=True, split="test")
print(livecodebench)

/opt/conda/envs/ro1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 880 examples [00:28, 30.90 examples/s] 

Dataset({
    features: ['question_title', 'question_content', 'platform', 'question_id', 'contest_id', 'contest_date', 'starter_code', 'difficulty', 'public_test_cases', 'private_test_cases', 'metadata'],
    num_rows: 880
})


In [4]:
livecodebench[0]

{'question_title': 'A. Short Sort',
 'question_content': 'There are three cards with letters $\\texttt{a}$, $\\texttt{b}$, $\\texttt{c}$ placed in a row in some order. You can do the following operation at most once: \n\n \n-  Pick two cards, and swap them.  Is it possible that the row becomes $\\texttt{abc}$ after the operation? Output "YES" if it is possible, and "NO" otherwise.\n\nInput\n\nThe first line contains a single integer $t$ ($1 \\leq t \\leq 6$)\xa0— the number of test cases.\n\nThe only line of each test case contains a single string consisting of each of the three characters $\\texttt{a}$, $\\texttt{b}$, and $\\texttt{c}$ exactly once, representing the cards.\n\nOutput\n\nFor each test case, output "YES" if you can make the row $\\texttt{abc}$ with at most one operation, or "NO" otherwise.\n\nYou can output the answer in any case (for example, the strings "yEs", "yes", "Yes" and "YES" will be recognized as a positive answer).Sample Input 1:\n6\n\nabc\n\nacb\n\nbac\n\nbca

In [5]:
from datetime import datetime

# deepseek paper uses lcb from 2024-08 to 2025-01
def is_date_in_range(date_str):
    date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S")
    # Define the start and end range
    start_date = datetime(2024, 8, 1)  # August 1, 2024
    end_date = datetime(2025, 2, 1)    # February 1, 2025

    # Check if the date is within the range
    return start_date <= date_obj < end_date

In [6]:
dataset = []
for entry in livecodebench:
    tests = entry["public_test_cases"]
    if isinstance(tests, str):
        try:
            tests = json.loads(tests)
        except json.JSONDecodeError as e:
            print(f"code reward Json Error parsing livecodebench: {e}")
            continue 

    if len(tests) == 0:
        continue 
    new_entry = {
        "problem": entry["question_content"],
        "starter_code": entry["starter_code"],
        "tests": tests,
    }
    if is_date_in_range(entry['contest_date']):
        dataset.append(new_entry)

print(f'Dataset size: {len(dataset)}')

output_dir = os.path.abspath("../../test/code")
output_file = os.path.join(output_dir, "livecodebench.json")

with open(output_file, "w") as f:
    json.dump(dataset, f, indent=4)


Dataset size: 279
